
# AI Skill Showcase — Tennis Project (Mingyu Fu)
My name is Mingyu Fu, I am currrently 1A student in honour mathematics. I have lots of experience on machine learning, such as number recognition system, tennis
player motion detection, natural language model, etc. I pick the one that I have done before, which is tennis player detection, this project is aiming to category
the motion of tennis player in either "Hitting" or "Other". Cause the data was collected manually, so the first step is to do something on the raw data.


## Environment

### If running on **Windwos || Linux**, consider installing following below:
```python
pip install numpy opencv-python pillow tqdm moviepy yolov5 torch torchvision
```


## Step 1 Detect / Preprocessing

In [ ]:

import cv2
import os
import torch
from pathlib import Path


class PicProcess():             #class for piacture preprocessing
    def __init__(self, model_path='ultralytics/yolov5', model_name='yolov5s'):
        self.model = torch.hub.load(model_path, model_name, pretrained=True)           #download yolov5s model fast and small           
    def Sequence2pic():
        vc = cv2.VideoCapture(r"D:/Tennis_Detect/Datasets(Vid2Pic)/RaW/202.mp4")         #vc stores the content of video
        n = 1  # count
        timeF = 10  # pic per fps (shannon rule)
        if vc.isOpened():
            rval, frame = vc.read()     #rval true if open succefully, frame store data
        else:
            rval = False
        i = 0
        while rval:
            rval, frame = vc.read()    #update rval status
            if (n % timeF == 0):  # per timeF fps store
                i += 1
                print(i)
                cv2.imwrite(r"D:/Tennis_Detect/Datasets(Vid2Pic)/RaW/2pic{}.jpg".format(i), frame)  # store
            n = n + 1
        vc.release()

    def pic2HalfGrey():
        #path = "C:\\Users\\25573\\Desktop\\Program\\Tennis_Player_Detect\\Datasets(Vid2Pic)"
        #files = os.listdir(path)
        i=int(1)
        for i in range(3996):
            if  os.path.exists("D:/Tennis_Detect/Datasets(Vid2Pic)/RaW/2pic"+str(i)+".jpg") == False:
                next
            else:
                img = Image.open("D:/Tennis_Detect/Datasets(Vid2Pic)/RaW/2pic"+str(i)+".jpg")
                width, height = img.size
                #img = img.convert('L')
                box = (0,height/2,width,986)       #986 because of audience may detected by yolo
                img = img.crop(box)
                img.save("D:/Tennis_Detect/Datasets(Vid2Pic)/After_Cut/Rf2Pic"+str(i)+".jpg")
                print("pic"+str(i)+"complete refine!")
    def Cut_Out_Athele(self, input_folder, output_folder):
        Path(output_folder).mkdir(parents=True, exist_ok=True)      #make sure every time run will replaced old picture
        for img_file in Path(input_folder).glob('*.jpg'):   #only seek the jpg picture in this directory
            image = cv2.imread(str(img_file))           #__str__
            if image is None:
                continue
            results = self.model(image)
            for i, det in enumerate(results.xyxy[0]):       #det = [x1, y1, x2, y2, conf, cls]
                if results.names[int(det[5])] == 'person':
                    xmin, ymin, xmax, ymax = map(int, det[:4])
                    cropped_img = image[ymin:ymax, xmin:xmax]
                    save_path = os.path.join(output_folder, f'{img_file.stem}_person_{i}.jpg')    #frame012.jpg → frame012
                    cv2.imwrite(save_path, cropped_img)



if __name__ == "__main__":
    PicProcess.pic2HalfGrey()

    input_folder = 'D:/Tennis_Detect/Datasets(Vid2Pic)/After_Cut/Temp'
    output_folder = 'D:/Tennis_Detect/Datasets(Vid2Pic)/After_Cut/Temp1'
    yolo = PicProcess()
    yolo.Cut_Out_Athele(input_folder, output_folder)



C:\Users\25573\AppData\Roaming\Python\Python313\site-packages\torch\hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\25573/.cache\torch\hub\master.zip


YOLOv5  2025-9-19 Python-3.13.5 torch-2.8.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:01<00:00, 11.2MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Using cache found in C:\Users\25573/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-9-19 Python-3.13.5 torch-2.8.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


## Step 2 Training

In [ ]:

import torch
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
#image size:224**224

train_dir = 'D:/Tennis_Detect/Datasets/Group(1) Picset/(1)ReadyPredictData(Hitting,no)/trial'
test_dir = 'D:/Tennis_Detect/Datasets/Group(1) Picset/(1)ReadyPredictData(Hitting,no)/test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = models.resnet50(pretrained=True)    #converge faster
num_ftrs = model.fc.in_features        #usually have 1000 but 2 now
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

weights = torch.tensor([2.0, 1.0], device=device)      #train data unbalanced so 2
criterion = nn.CrossEntropyLoss(weight=weights)

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5) #1e-5 avoid overfittng
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) #time 0.1 per 5 epoch

def evaluate_model(model, test_loader, device):
    model.eval()        #stop dropout
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1) #take the most possible one
            all_preds.extend(predicted.cpu().numpy())   #flat in cpu as numpy only on cpu
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(confusion_matrix(all_labels, all_preds))
    print(classification_report(all_labels, all_preds))
    return accuracy

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")
    test_accuracy = evaluate_model(model, test_loader, device)
    print(f"Test Accuracy: {test_accuracy}")

    torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image


MODEL_WEIGHTS = 'D:/Program/Pytorch_Status_Detect.pth'
CLASS_NAMES = ["no_hitting", "hitting"] #0，1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_model(weights_path: str):
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    state = torch.load(weights_path, map_location=device)
    model.load_state_dict(state)
    model.to(device)
    model.eval()
    return model

@torch.no_grad()
def predict_image(model, img_path: str):
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)  # [1,3,224,224] add batch dimension
    logits = model(x)
    probs = torch.softmax(logits, dim=1).squeeze(0).cpu().tolist()    #0~1
    pred_idx = int(torch.argmax(logits, dim=1).item())
    pred_label = CLASS_NAMES[pred_idx] if pred_idx < len(CLASS_NAMES) else str(pred_idx) ##
    return pred_label, probs[pred_idx], probs 

if __name__ == "__main__":
    
    IMG_PATH = r"D:/Program/RfPic2_person_0.jpg"
    model = load_model(MODEL_WEIGHTS)
    label, conf, all_probs = predict_image(model, IMG_PATH)
    print(f"Image: {IMG_PATH}")
    print(f"Pred: {label}  (confidence={conf:.4f})")
    print("All probs:", {CLASS_NAMES[i]: round(p, 4) for i, p in enumerate(all_probs)})

    IMG_PATH = r"D:/Program/RfPic168_person_0.jpg"
    label, conf, all_probs = predict_image(model, IMG_PATH)
    print(f"Image: {IMG_PATH}")
    print(f"Pred: {label}  (confidence={conf:.4f})")
    print("All probs:", {CLASS_NAMES[i]: round(p, 4) for i, p in enumerate(all_probs)})


Image: D:/Program/RfPic2_person_0.jpg
Pred: no_hitting  (confidence=0.9999)
All probs: {'no_hitting': 0.9999, 'hitting': 0.0001}
Image: D:/Program/RfPic168_person_0.jpg
Pred: hitting  (confidence=1.0000)
All probs: {'no_hitting': 0.0, 'hitting': 1.0}



## Results & Discussion
- As a result, this model could succefully detect the tennis player either is hitting the ball or not hitting the ball, which help tennis player to evaulate their
performance after match.

<div style="display:grid; grid-template-columns: repeat(2, 1fr); gap:14px; align-items:start; max-width:300px;">
  <figure style="margin:0; padding:10px; border:1px solid #e5e7eb; border-radius:14px; box-shadow:0 2px 10px rgba(0,0,0,.04);">
    <figcaption style="font-weight:600; margin-bottom:8px; font-size:14px;">Expected: 1 · Output: 1</figcaption>
    <img src="./RfPic168_person_0.jpg" style="width:260px; height:auto; border-radius:10px; display:block;">
  </figure>
  <figure style="margin:0; padding:10px; border:1px solid #e5e7eb; border-radius:14px; box-shadow:0 2px 10px rgba(0,0,0,.04);">
    <figcaption style="font-weight:600; margin-bottom:8px; font-size:14px;">Expected: 0 · Output: 0</figcaption>
    <img src="./RfPic2_person_0.jpg" style="width:260px; height:auto; border-radius:10px; display:block;">
  </figure>
</div>

The full concept of the project was describe in an article: https://doi.org/10.54254/2755-2721/2025.21197. You can see the origin source code in: https://github.com/Jerry-code-oss/Luck_Program/tree/main/Tennis_Detect/Code

